# Customizing Federated Computations

In this next part of the tutorial it will be up to you to customize the `BaseModelOwner` or the `BaseDataOwner` to implement a new way of computing the gradients and securely aggregating them.

### Boilerplate

First up is the boilerplate code from the previous part. This includes configuring TF Encrypted and importing all of the dependencies. We've removed the `default_model_fn` and `secure_mean` functions as you'll write new version of those.

In [3]:
import tensorflow as tf
import tf_encrypted as tfe

players = [
    'server0', 
    'server1', 
    'crypto-producer', 
    'model-owner',
    'data-owner-0',
    'data-owner-1',
    'data-owner-2',
]
config = tfe.EagerLocalConfig(players)

tfe.set_config(config)
tfe.set_protocol(tfe.protocol.Pond())

from players import BaseModelOwner, BaseDataOwner
from func_lib import default_model_fn, secure_mean, evaluate_classifier
from util import split_dataset
from download import download_mnist

NUM_DATA_OWNERS = 3
BATCH_SIZE = 256
DATA_ITEMS = 60000
BATCHES = DATA_ITEMS // NUM_DATA_OWNERS // BATCH_SIZE

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/justinpatriquin/projects/tf-encrypted/tf_encrypted/operations/secure_random/secure_random_module_tf_2.0.0.so'


### Implementing Reptile Meta-Learning Algorithm

In this section you will use the information from the previous tutorial to help implement new functions for the `model_fn` and the `aggregator_fn`. We also recommend checking out the implementations of `default_model_fn` and `secure_mean` located in [func_lib.py](./func_lib.py) for some help figuring out where to start. 

We've done this with reptile and recommend following through with this but if you have another idea feel free to implement that!

**TODO**: might need to add details to the below paragraph

The reptile meta-learning algorithm computes k steps of SGD. When paired with the secure_aggregation aggregator_fn, this model_fn corresponds to using g_k as the outer gradient update. See the Reptile paper for more: https://arxiv.org/abs/1803.02999

**HINT**: For implementing reptile it'll help to take advantage of the already implemented `default_model_fn` and `secure_mean` to use inside of the functions below.

In [ ]:
def reptile_model_fn(data_owner, iterations=3,
                     grad_fn=default_model_fn, **kwargs):
  #TODO
  return [var.read_value() for var in data_owner.model.trainable_variables]

In [ ]:
def secure_reptile(collected_inputs, model):
  # TODO
  return weights_deltas

### Customize Base Classes

In [ ]:
class ModelOwner(BaseModelOwner):
  @classmethod
  def model_fn(cls, data_owner):
    # TODO

  @classmethod
  def aggregator_fn(cls, model_gradients, model):
    # TODO

  @classmethod
  def evaluator_fn(cls, model_owner):
    return evaluate_classifier(model_owner)

In [4]:
# TODO its not super clear how DataOwner should come into the picture here when customizing ModelOwner is sufficient
class DataOwner(BaseDataOwner):
  pass

### Continue Boilerplate

In this section we continue the fill in some of the boilerplate code from the previous tutorial.

In [ ]:
download_mnist()
split_dataset("./data", NUM_DATA_OWNERS, DATA_ITEMS)

model = tf.keras.Sequential((
    tf.keras.layers.Dense(512, input_shape=[None, 28 * 28],
                          activation='relu'),
    tf.keras.layers.Dense(10),
))

model.build()

loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(LEARNING_RATE)

model_owner = ModelOwner("model-owner",
                         "{}/train.tfrecord".format("./data"),
                         model, loss,
                         optimizer=opt)

In this next part consider how you might use another learning rate to customize the reptile training loop.

In [5]:
# Simplify this with a loop?
data_owners = [DataOwner("data-owner-{}".format(i),
                         "{}/train{}.tfrecord".format("./data", i),
                         model, loss,
                         optimizer=opt)
               for i in range(NUM_DATA_OWNERS)]

NameError: name 'DataOwner' is not defined

Now train! Remember we're using TensorFlow 2.0 so it should be  easy to explore the computations and see the actual values being passed around in the computations. You can use this to help debug any problems run into while implementing the reptile meta-learning algorithm.

In [ ]:
model_owner.fit(data_owners, rounds=BATCHES, evaluate_every=10)

print("\nDone training!!")

Check out the solution [here](./b%20-%20Customizing%20Federated%20Computations%20-%20Solution.ipynb) and compare it to what you've accomplished.